<a href="https://colab.research.google.com/github/drf0rk/AnxLight/blob/main/notebook/AnxLight_Launcher_v0.0.8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Cell 1: AnxLight v3 Pre-Flight Setup
# This cell ensures the repository is up-to-date and runs the main environment setup.

import os
import subprocess
import sys

print("--- AnxLight v3 Pre-Flight Setup Cell ---")

# --- Step 1: Clone/Update the Repository ---
repo_path = '/content/AnxLight'
if not os.path.exists(repo_path):
    print("Cloning AnxLight repository...")
    try:
        subprocess.run(['git', 'clone', 'https://github.com/drf0rk/AnxLight.git', repo_path], check=True)
    except subprocess.CalledProcessError as e:
        print(f"FATAL: Failed to clone repository. Error: {e}")
        sys.exit(1) # Stop execution
else:
    print("AnxLight repository already exists. Pulling latest changes...")
    try:
        # Discard any local changes and pull the latest from the repo for a clean state
        subprocess.run(['git', '-C', repo_path, 'reset', '--hard', 'origin/main'], check=True)
        subprocess.run(['git', '-C', repo_path, 'pull'], check=True)
    except subprocess.CalledProcessError as e:
        print(f"WARNING: Failed to pull latest changes. Error: {e}")


# --- Step 2: Change Directory and Set Environment ---
os.chdir(repo_path)
print(f"Current working directory: {os.getcwd()}")
os.environ['PROJECT_ROOT'] = repo_path
print(f"Environment variable PROJECT_ROOT set to: {os.environ['PROJECT_ROOT']}")


# --- Step 3: Execute the Pre-Flight Setup Script ---
project_root = os.environ.get('PROJECT_ROOT', repo_path)
pre_flight_script_path = os.path.join(project_root, 'scripts', 'pre_flight_setup.py')
python_executable = sys.executable

if not os.path.exists(pre_flight_script_path):
    print(f"FATAL: The pre-flight setup script was not found at {pre_flight_script_path}")
    sys.exit(1)

print(f"\n--- Executing Pre-Flight Setup Script ---")
print(f"Running command: {python_executable} {pre_flight_script_path}")
print("-" * 40)

try:
    subprocess.run([python_executable, pre_flight_script_path], check=True)
    print("-" * 40)
    print("--- Pre-Flight Setup Script Finished Successfully ---")
except subprocess.CalledProcessError as e:
    print("-" * 40)
    print(f"FATAL: Pre-Flight Setup Script failed with return code {e.returncode}.")
    print("Please check the logs above for errors. Cell 2 will likely fail.")
    sys.exit(1)

--- AnxLight v3 Pre-Flight Setup Cell ---
AnxLight repository already exists. Pulling latest changes...
Current working directory: /content/AnxLight
Environment variable PROJECT_ROOT set to: /content/AnxLight

--- Executing Pre-Flight Setup Script ---
Running command: /usr/bin/python3 /content/AnxLight/scripts/pre_flight_setup.py
----------------------------------------


In [ ]:
# @title Cell 2: AnxLight v3 Gradio App Launcher (Keeps Process Alive)
# This cell launches the main Gradio UI and then waits, keeping the process alive.

import os
import sys
import subprocess
import time

print("--- AnxLight v3 Launch Cell ---")

project_root = os.environ.get('PROJECT_ROOT', '/content/AnxLight')
venv_python_path = os.path.join(project_root, 'anxlight_venv', 'bin', 'python')
main_gradio_script_path = os.path.join(project_root, 'scripts', 'main_gradio_app.py')

if not all(os.path.exists(p) for p in [project_root, venv_python_path, main_gradio_script_path]):
    print(f"\n[CRITICAL ERROR] A required file or directory was not found. Please ensure Cell 1 ran successfully.")
    sys.exit(1)

gradio_env = os.environ.copy()
gradio_env['PYTHONPATH'] = f"{os.path.join(project_root, 'modules')}:{project_root}:{os.path.join(project_root, 'scripts')}:{gradio_env.get('PYTHONPATH', '')}"

print("\n--- Launching Gradio Application Subprocess ---")
print(f"Executing: {venv_python_path} {main_gradio_script_path}")

gradio_process = subprocess.Popen(
    [venv_python_path, main_gradio_script_path],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
    universal_newlines=True,
    env=gradio_env,
    cwd=project_root
)

print("\n--- Waiting for Gradio to launch and provide a public URL... ---")

# Monitor for the URL
url_found = False
for line in iter(gradio_process.stdout.readline, ''):
    print(line, end='')
    if "Running on public URL:" in line:
        print("\n" + "="*60)
        print("          GRADIO IS RUNNING AND READY!")
        print("          Click the public URL above to open the UI.")
        print("\n          This cell will continue to run to keep the app alive.")
        print("          Stop this cell manually when you are finished.")
        print("="*60 + "\n")
        url_found = True
        break # Exit the initial monitoring loop

# If the URL was found, wait for the process to terminate.
# This will keep the cell running and the Gradio app alive.
if url_found:
    # Now, stream the rest of the output indefinitely
    for line in iter(gradio_process.stdout.readline, ''):
        print(line, end='')
    # The script will wait here until the process is manually stopped.
    gradio_process.wait()
else:
    print("\n[ERROR] The Gradio process finished before a public URL was generated. Check logs for errors.")